# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Web services

This notebook contains Kusto queries that can help getting to the root cause of an issue with web services for an environment. 

Each section in the notebook contains links to relevant documentation from the performance tuning guide [aka.ms/bcperformance](aka.ms/bcperformance),   telemetry documentation in [aka.ms/bctelemetry](aka.ms/bctelemetry), as well as Kusto queries that help dive into a specific area.

NB! Some of the signal used in this notebook is only available in newer versions of Business Central, so check the version of your environment if some sections do not return any data. The signal documentation states in which version a given signal was introduced.

**NB!** Telemetry for SOAP endpoints does not emit HTTP status code. So the sections that query for different values of HTTP status will not show results for these requests.

## 1. Connect to Application Insights
First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [27]:
# load the KQLmagic module
%reload_ext Kqlmagic

# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'

## 2. Define filters
This workbook is designed for troubleshooting a single environment. Please provide values for aadTenantId and environmentName: 

In [28]:
aadTenantId = "<Add AAD tenant id here>"
environmentName = "<add environment name here>"

# Analyze web service usage
Now you can run Kusto queries to look for possible root causes for issues about web services.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

## Web service requests overview

Performance tuning guide: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#writing-efficient-web-services

Web service telemetry docs: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace

KQL sample: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/WebServiceCalls.kql

In [29]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
    and timestamp > ago(7d)
| extend category = tostring( customDimensions.category )
| summarize request_count=count() by category, bin(timestamp, 1d)
| render timechart title= 'Number of web service requests by category'

[{'category': 'Api', 'timestamp': datetime.datetime(2020, 9, 17, 0, 0, tzinfo=tzutc()), 'request_count': 7},
 {'category': 'Api', 'timestamp': datetime.datetime(2020, 9, 24, 0, 0, tzinfo=tzutc()), 'request_count': 5}]

In [30]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
    and timestamp > ago(7d)
| extend category = tostring( customDimensions.category )
       , executionTimeInMS = toreal(totimespan(customDimensions.serverExecutionTime))/10000 //the datatype for executionTime is timespan 
| summarize count=count() by executionTime_ms = bin(executionTimeInMS, 100), category
| order by category, executionTime_ms asc
| render columnchart with  (ycolumns = count, series = category, title= 'Execution time (in milliseconds) of web service requests by category' ) 


[{'executionTime_ms': 0.0, 'category': 'Api', 'count': 9},
 {'executionTime_ms': 200.0, 'category': 'Api', 'count': 3}]

## Web service throttling
Operational Limits for Business Central Online: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/operational-limits-online#query-limits

Telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace

In [31]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
    and timestamp > ago(7d)
| extend httpStatusCode = tostring( customDimensions.httpStatusCode )
| summarize count() by bin(timestamp, 1d), httpStatusCode
| render timechart title= 'Number of web service requests by http status code'

[{'timestamp': datetime.datetime(2020, 9, 17, 0, 0, tzinfo=tzutc()), 'httpStatusCode': '200', 'count_': 7},
 {'timestamp': datetime.datetime(2020, 9, 24, 0, 0, tzinfo=tzutc()), 'httpStatusCode': '200', 'count_': 5}]

## Web service requests (Access denied)
The user who made the request doesn't have proper permissions. For more information, see 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/webservices/web-services-authentication
* https://docs.microsoft.com/en-us/dynamics365/business-central/ui-define-granular-permissions

Telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace

In [32]:
%%kql
//
// Top 10 endpoint requests with access denied
//
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
    and timestamp > ago(7d)
    and customDimensions.httpStatusCode == '401'
| limit 10

,timestamp,Precisetimestamp,env_cloud_name,env_cloud_environment,ManagementService,ManagementTenant,env_cloud_location,env_cloud_deploymentUnit,AppServiceId,env_cloud_role,env_cloud_roleInstance,Level,ProviderGuid,ProviderName,EventId,Pid,Tid,OpcodeName,KeywordName,TaskName,ChannelName,ActivityId,appInsightstimestamp,message,itemType,severity,customDimensions,operation_Id,operation_Name,operation_ParentId,operation_SyntheticSource,session_Id,iKey,SourceNamespace,SourceMoniker,SourceVersion


[]

## Web service requests (Not found)
The given endpoint was not valid

See
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/webservices/publish-web-service

Telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace

In [33]:
%%kql
//
// Top 10 non-valid endpoints called
//
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
    and timestamp > ago(7d)
    and customDimensions.httpStatusCode == '404'
| summarize number_of_requests=count() by endpoint = tostring( customDimensions.endpoint ), alObjectName = tostring( customDimensions.alObjectName ), alObjectId = tostring( customDimensions.alObjectId )
| order by number_of_requests desc
| limit 10

,endpoint,alObjectName,alObjectId,number_of_requests


[]

## Web service requests (Request timed out)
The request took longer to complete than the threshold configured for the service

See
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/operational-limits-online#ODataServices

Telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace

Performance tuning guide (you need to tune these endpoints to make them go faster)
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#writing-efficient-web-services

In [34]:
%%kql
//
// Top 10 endpoints that times out
//
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
    and timestamp > ago(7d)
    and customDimensions.httpStatusCode == '408'
| summarize number_of_requests=count() by endpoint = tostring( customDimensions.endpoint ), alObjectName = tostring( customDimensions.alObjectName ), alObjectId = tostring( customDimensions.alObjectId )
| order by number_of_requests desc
| limit 10

,endpoint,alObjectName,alObjectId,number_of_requests


[]

## Web service requests (Too Many Requests)
The request exceeded the maximum simultaneous requests allowed on the service.

See
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/operational-limits-online#ODataServices

Telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace

Performance tuning guide (you need to make your web service client backoff and retry)
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#writing-efficient-web-services

In [35]:
%%kql
//
// Top 10 endpoints get throttled
//
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
traces
| where 1==1 
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
    and timestamp > ago(7d)
    and customDimensions.httpStatusCode == '426'
| summarize number_of_requests=count() by endpoint = tostring( customDimensions.endpoint ), alObjectName = tostring( customDimensions.alObjectName ), alObjectId = tostring( customDimensions.alObjectId )
| order by number_of_requests desc
| limit 10

,endpoint,alObjectName,alObjectId,number_of_requests


[]